<a href="https://colab.research.google.com/github/edogriba/FOR-2024/blob/main/LAB01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

In [5]:
# We need to import the package mip
import mip

1. Sets

- $I$ = set of food

- $J$ = set of nutrients

2. Parameters

- $c_i$ = cost of food $i \in I$

- $f_i$ = availability of food $i \in I$

- $b_j$ = min requirement of nutrient $j \in J$

- $a_{ij}$ = amount of nutrient $j \in J$ in food $i \in I$

3. Decision variables

- $x_i$ = amount of food $i \in I$

4. Objective function

- min $\sum_{i}c_i x_i$

5. Constraints  such that:

- $\sum_{i=1}^n a_{ij}x_i >= b_j$ for every $j \in J$ (min requirement)

- $x_i <= f_i$ for every $i \in I$ (availability)

- $x_i >= 0$


In [ ]:
# Dictionaries

# D = { key: value }
# L = {key}

In [2]:
# Food --> Set of food items
I = {'Bread', 'Milk', 'Eggs', 'Meat', 'Cake'}

# Nutrients --> Set of nutrients
J = {'Calories', 'Proteins', 'Calcium'}

# Cost in Euro per hg of food
c = {'Bread': 0.1, 'Milk':0.5, 'Eggs': 0.12,'Meat': 0.9, 'Cake': 1.3}

# Availability per hg of food
q = {'Bread': 4, 'Milk':3, 'Eggs': 1,'Meat': 2, 'Cake': 2}

# minum nutrients
b = {'Calories': 600, 'Proteins': 50, 'Calcium': 0.7}


# Nutrients per hf of food
a = {('Bread','Calories'):30,('Milk','Calories'):50,('Eggs','Calories'):150,('Meat','Calories'):180,('Cake','Calories'):400,
('Bread','Proteins'):5,('Milk','Proteins'):15,('Eggs','Proteins'):30,('Meat','Proteins'):90,('Cake','Proteins'):70,
('Bread','Calcium'):0.02,('Milk','Calcium'):0.15,('Eggs','Calcium'):0.05,('Meat','Calcium'):0.08,('Cake','Calcium'):0.01}

In [3]:
for i in I:
    print(c[i])

0.12
0.9
1.3
0.1
0.5


Now we create an empty model and add the variables

In [6]:
# Define a model
model = mip.Model() # create an empty model

# Define variables --> we'll create them as dictionaries
## x = [model.add_var(name = i,lb=0) for i in I]
x = {i: model.add_var(name = i) for i in I} # name --> gives a name lb stands for lower bound, both are optional arguments

Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [7]:
# Define the objective function
model.objective = mip.minimize(mip.xsum(c[i]*x[i] for i in I))

In [ ]:
# List x = [[], [], []]
# Dict x[,]

The constraints can be generated in loops:

In [8]:
# CONSTRAINTS

# Availability constraint
for i in I:
  model.add_constr(x[i] <= q[i])

# Minimum of nutrients
for j in J:
  model.add_constr(mip.xsum(a[i, j]*x[i] for i in I) >= b[j])

The model is complete. Let's solve it and print the optimal solution.

In [9]:
# Optimizing command
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [10]:
# Optimal objective function value
model.objective.x

3.37

In [19]:
# Printing the variables values
print("We use: ")
for i in model.vars:
  print(f"{i.x:.1f} {i.name}")


We use: 
1.0 Eggs
1.5 Meat
0.0 Cake
4.0 Bread
3.0 Milk


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


1. Sets

- $I$ = types of oil
- $J$ = types of gazoline

2. Parameters

- $c_i$ = cost of oil $i \in I$
- $q_i$ = availability of oil $i \in I$
- $r_j$ = price of gazoline $j \in J$
- $q_{min,ij}$ = minimum amount of oil $i \in I$ in gazoline $j \in J$
- $q_{max,ij}$ = maximum amount of oil $i \in I$

3. Decision Variables

- $x_{ij}$ = amount of oil $i \in I$ that goes into gazoline $j \in J$
- $y_j$ = amount of gazoline $j \in J$

4. Objective function

- max $\sum_{j}(r_j  y_j)- \sum_{i}(\sum_{j}(c_i x_{ij}))$

5. Constraints such that:

- $\sum_{i}x_{ij}\leq q_i$ for every $i \in I$

- $\sum_{i}x_{ij} = y_j$ for every $j \in J$

- ${x_{ij} \over y_{ij}} \geq q_{min, ij}$ for every $i \in I$ and for every $j \in J$

- ${x_{ij} \over y_{ij}} \leq q_{max, ij}$ for every $i \in I$ and for every $j \in J$

- $x_{ij}, y_{ij} \geq 0$


In [21]:
# Set of oil types
I = {1, 2, 3, 4}

# Set of gasoline types
J = {"A", "B", "C"}

# Unit cost for oil of type i
c = {1: 9, 2: 7, 3: 12, 4: 6}

# Availability of oil type i
b = {1: 5000, 2: 2400, 3: 4000, 4: 1500}

# Price of gasoline of type j
r = {'A': 12, 'B': 18, 'C': 10}

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1
q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0
q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

Now we create an empty model and add the variables

In [24]:
# Define a model
model2 = mip.Model()

# Define variables
x = {(i, j): q_min[(a, b)] for a in I for b in J}
y = {j: e for e in J}

KeyError: (1, 'B')

In [ ]:
# Define the objective function
model2.objective = mip.maximize(mip.xsum()-mip.xsum())

In [ ]:
# CONSTRAINTS
# Availability constraint
for i in I:
  model2.add_constr()

# Conservation constraint
for j in J:
  model2.add_constr()

# Maximum quantity
for i in I:
  for j in J:
    model2.add_constr()

# Minimum quantity
for i in I:
  for j in J:
    model2.add_constr()

The model is complete. Let's solve it and print the optimal solution.

In [ ]:
# Optimizing command
model2.optimize()

In [ ]:
# Optimal objective function value
model2.objective.x

In [ ]:
# Printing the variables values
for i in model2.vars:
  print(i.name)
  print(i.x)
  print('-----')